In [2]:
import pandas as pd

# Load the dataset
data = pd.read_csv('creditcard.csv')

# Inspect the dataset
print(data.head())  # First 5 rows
print(data.info())  # Column names, data types, and missing values
print(data['Class'].value_counts())  # Distribution of fraud vs non-fraud cases

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [3]:
from sklearn.preprocessing import StandardScaler

# Normalize the 'Amount' column
scaler = StandardScaler()
data['Amount'] = scaler.fit_transform(data['Amount'].values.reshape(-1, 1))

# Drop the 'Time' column (not useful for modeling)
data = data.drop('Time', axis=1)

# Verify the dataset after preprocessing
print(data.head())

         V1        V2        V3        V4        V5        V6        V7  \
0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9       V10  ...       V21       V22       V23       V24  \
0  0.098698  0.363787  0.090794  ... -0.018307  0.277838 -0.110474  0.066928   
1  0.085102 -0.255425 -0.166974  ... -0.225775 -0.638672  0.101288 -0.339846   
2  0.247676 -1.514654  0.207643  ...  0.247998  0.771679  0.909412 -0.689281   
3  0.377436 -1.387024 -0.054952  ... -0.108300  0.005274 -0.190321 -1.175575   
4 -0.270533  0.817739  0.753074  ... -0.009431  0.798278 -0.137458  0.141267   

        V25       V26       V27       V28    Amount  Class  
0  0.12

In [4]:
# Features (X) and target (y)
X = data.drop('Class', axis=1)  # All columns except 'Class'
y = data['Class']  # The 'Class' column (fraud or not)

print(X.shape, y.shape)  # Check the shapes of X and y

(284807, 29) (284807,)


In [5]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Verify the resampled data
print(y_resampled.value_counts())  # Should have equal fraud and non-fraud cases

Class
0    284315
1    284315
Name: count, dtype: int64


In [6]:
from sklearn.model_selection import train_test_split

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Verify the shapes
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(454904, 29) (113726, 29) (454904,) (113726,)


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56750
           1       1.00      1.00      1.00     56976

    accuracy                           1.00    113726
   macro avg       1.00      1.00      1.00    113726
weighted avg       1.00      1.00      1.00    113726

Accuracy: 0.9998944832316269


In [8]:
import joblib

# Save the model
joblib.dump(rf_model, 'fraud_detection_model.pkl')

# Save the scaler (for preprocessing new data)
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [12]:
import streamlit as st
import joblib
import numpy as np

# Load the model and scaler
model = joblib.load('fraud_detection_model.pkl')
scaler = joblib.load('scaler.pkl')

# Streamlit app
st.title("Fraud Detection in Financial Transactions")

# Input fields for features
st.header("Enter Transaction Details")
input_data = []
for i in range(1, 29):
    input_data.append(st.number_input(f"V{i}", value=0.0))

amount = st.number_input("Amount", value=0.0)

# Preprocess the input data
input_data = np.array(input_data).reshape(1, -1)
amount_scaled = scaler.transform([[amount]])

# Combine features
features = np.hstack((input_data, amount_scaled))

# Make prediction
if st.button("Predict"):
    prediction = model.predict(features)
    if prediction[0] == 1:
        st.error("Fraudulent Transaction Detected!")
    else:
        st.success("Legitimate Transaction.")

2025-01-28 15:38:27.418 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-28 15:38:27.419 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-28 15:38:27.419 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-28 15:38:27.419 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-28 15:38:27.420 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-28 15:38:27.420 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-28 15:38:27.420 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-28 15:38:27.421 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar